In [ ]:
# default_exp signUp

# SignUp

> Interface to sign up user

In [ ]:
#hide
from nbdev.showdoc import *
import src.setEnv 
import requests

In [ ]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from beartype import beartype
from uuid import uuid4
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from time import time
from functools import cached_property
import json

In [ ]:
#check whether hash password is working here
examplePassword1 = '123'
exampleHashedPassword1 = hash_password(examplePassword1)
exampleHashedPassword1

'0547b2f5f51a527117ecdfef68b5409fc3fda174d8c47d93bcdd4d67aa6c0d83:51b1558fc1fe46f180498dd98cd98d07'

## User Input Class

In [ ]:
#export 
@dataclass_json
@dataclass
class UserInput:
    username: str
    password: str
    email: str 
    
    @cached_property
    def passwordHash(self):
        return hash_password(self.password)
    
    def saveTable(self):
        passwordHashed = self.passwordHash
        userTable = UserTable(
                userId = str(uuid4()),
                username = self.username,
                date = time(),
                passwordHash = passwordHashed,
                email = self.email
            )
        userTable.save()
        

In [ ]:
example = UserInput.from_dict({'username': 'DelG', 'password': '123', 'email': 'chanopsethiwan233@gmail.com'})
example

UserInput(username='DelG', password='123', email='chanopsethiwan233@gmail.com')

## Main Lambda Handler

In [ ]:
#export 
class H:
    class ParseInputError(Exception): pass
    class SavingError(Exception): pass
    class CountUsernameError(Exception): pass
    class UsernameAlreadyExistError(Exception): pass
    class CountEmailError(Exception):pass
    class EmailAlreadyExistError(Exception):pass
    @classmethod
    @beartype
    def parseInput(cls, event:dict)->UserInput:
        try:
            user = Event.parseDataClass(UserInput, deepcopy(event))
            return user
        except Exception as e:
            raise cls.ParseInputError(e)
    
    @classmethod
    @beartype
    def saveUserMethod(cls, user:UserInput)->bool:
        try:
            usernameExist = UserTable.username_index.count(user.username) > 0
        except Exception as e:
            raise cls.CountUsernameError(e)
        try: 
            emailExist = UserTable.email_index.count(user.email) > 0
        except Exception as e:
            raise cls.CountEmailError(e)
            
        if usernameExist: raise cls.UsernameAlreadyExistError
        if emailExist: raise cls.EmailAlreadyExistError
            
        try:
            user.saveTable()
            return True
        except Exception as e:
            raise cls.SavingError(e)

    
        

In [ ]:
#export
def signUp(event, *args):
    try:
        user = H.parseInput(event)
        H.saveUserMethod(user)
        return Response.returnSuccess()
    except H.ParseInputError as e:
        return Response.returnError(f'failed to parse input {e}')
    except H.SavingError as e:
        return Response.returnError(f'failed saving user {e}')
    except H.CountUsernameError as e:
        return Response.returnError(f'failed to count number of user {e}')
    except H.UsernameAlreadyExistError as e:
        return Response.returnError(f'username already exist {e}')
    except H.CountEmailError as e:
        return Response.returnError(f'failed to count number of email {e}')
    except H.EmailAlreadyExistError as e:
        return Response.returnError(f'email already exist {e}')
    except Exception as e:
        return Response.returnError(f' unknown error {e}')
    

In [ ]:
input_ = {'username': 'Delg45678', 'password': '12764', 'email': 'chanopsethiwan20@gmail.com'}
event = Event.getInput(body = input_)
signUp(event)

{'body': '{}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [ ]:
epoch = time()
epoch

1662460127.619107

In [ ]:
a = Event.getInput({'username': 'DelG197', 'password': '123456'})
json.dumps(a)

'{"body": "{\\"username\\":\\"DelG197\\",\\"password\\":\\"123456\\"}", "httpMethod": null, "multiValueHeaders": null, "multiValueQueryStringParameters": null, "path": null, "pathParameters": null, "queryStringParameters": {}, "requestContext": null, "headers": {}, "statusCode": 200, "isBase64Encoded": false}'

In [ ]:
a = ''
if not a:
    print('false')
else:
    print('true')

false


In [ ]:
print(not False)

True


In [ ]:
url = 'https://3d4v07deqh.execute-api.ap-southeast-1.amazonaws.com/Prod/signup'
body = {'username': 'delg', 'password': 'abcd', 'email':'sajkdsfdjka' }
r = requests.post(url, json=body)
print(r.status_code)
print(r.text)

403
{"message":"Missing Authentication Token"}


In [ ]:
'https://3d4v07deqh.execute-api.ap-southeast-1.amazonaws.com/Prod/signup'